In [30]:
import pandas as pd 
from pyvi import ViTokenizer
import re
import os
import string

In [31]:
path = os.getcwd()
path_raw = os.path.join(path,'raw_data/raw_data_label.csv')
path_clean = os.path.join(path,'clean_data/cleaned_data.csv')

In [32]:
data_after = pd.read_csv(path_raw)
data_before = pd.read_csv(path_raw)

In [17]:
replace_list ={'oà': 'òa', 'oá': 'óa', 'oả': 'ỏa', 'oã': 'õa', 'oạ': 'ọa', 'oè': 'òe', 'oé': 'óe','oẻ': 'ỏe',
        'oẽ': 'õe', 'oẹ': 'ọe','uá':'úa','uà':'ùa','uả':'ủa','uã':'ũa','uạ':'ụa', 'uỳ': 'ùy', 'uý': 'úy', 'uỷ': 'ủy', 'uỹ': 'ũy','uỵ': 'ụy', 'uả': 'ủa',
        'ả': 'ả', 'ố': 'ố', 'u´': 'ố','ỗ': 'ỗ', 'ồ': 'ồ', 'ổ': 'ổ', 'ấ': 'ấ', 'ẫ': 'ẫ', 'ẩ': 'ẩ',
        'ầ': 'ầ', 'ỏ': 'ỏ', 'ề': 'ề','ễ': 'ễ', 'ắ': 'ắ', 'ủ': 'ủ', 'ế': 'ế', 'ở': 'ở', 'ỉ': 'ỉ',
        'ẻ': 'ẻ', 'àk': u' à ','aˋ': 'à', 'iˋ': 'ì', 'ă´': 'ắ','ử': 'ử', 'e˜': 'ẽ', 'y˜': 'ỹ', 'a´': 'á',
        #emoji -> text   
        '😔': ' negative ','😡': ' negative ','⭐': ' positive ','😍': ' positive ','🙂': ' positive ',
        '😤': ' negative ','🤣': ' positive ','😡': ' negative ','🤗': ' positive ','😁': ' positive ',
        '🌼': ' positive ','😚': ' positive ','😅': ' positive ','😞': ' negative ','😀': ' postive ',
        '👍': ' positive ','💯': ' positive ','😥': ' negative ','❤': ' positive ','😱': ' negative ',
        '😎': ' positive ','👌': ' positive ','🤬': ' negative ','😋': ' positive ','😄': ' positive ',
        '😖': ' negative ','😊': ' positive ',
        #Chuẩn hóa sentiment word
        ':))': ' positive ', ':)': ' positive ', ':v':' positive ','“': ' ',
        'bit':' biết ','bik':' biết ','bk':' biết ','bít':' biết ','biet':' biết ','beautiful': ' đẹp tuyệt vời','bin':' pin ','bt':' bình thường ',
        ' co ':' có ',' cam ':' camera ',' came ':' camera ','cmr':' camera ','chs ':' chơi ','choi':' chơi ','củng':' cũng ','cung':' cũng ','chất lg':' chất lượng ',
        'cx':' cũng ','cungx':' cũng ','chiệu':' chịu ','chăc':' chắc ','chac':' chắc ','chắt':' chắc ',
        'đmx':' điện máy xanh ','dmx':' điện máy xanh ','dung':' dùng ','đỗi':' đổi ','dẹp':' đẹp ','đep':' đẹp ','dây':' đây ',
        'đx':' được ','dk':' được ','dc':' được ','đc':' được ',' đ ':' được ',' đả ':' đã ','đt ':' điện thoại ','dt':' điện thoại ','đth ':' điện thoại ',
        ' e ':' em ','fb ':' facebook ','face':' facebook ','flat':' flash ','plash':' flash ',
        'gét':' ghét ','ghet':' ghét ',' gi ':' gì ','gem':' game ','gud':' tốt ','god':' tốt ','good':' tốt ','gút':' tốt ',
        'kg ':' không ','not':' không ',' k ':' không ',' kh ':' không ',' kô ':' không ',' ko ':' không ',
        'khong':' không ','khoản ':' khoảng ','khuyến':' khuyên ','kem':' kém ',
        ' hok ':' không ','hìh':' hình ','he he':' positive ','hehe':' positive ','hihi':' positive ','haha':' positive ',
        'hjhj':' positive ','huhu':' nagative ','hàg': ' hàng ',   
        'luân':' luôn ','lun':' luôn ','luon':' luôn ','lác':' lag ','lad':' lag ','lắc':' lag ','lát':' lag ','lang':' lag ',
        'lau':' lâu ','loi':' lỗi ','lổi':' lỗi ','lét':' nét ',
        'mik':' mình ','minh':' mình ','mk ':' mình ',' m ':' mình ','mak':' mà ',' ma ':' mà ','mấy':' máy ','may':' máy ','mạn ':' mạng ','mang':' mạng ',
        'mn':' mọi người ','mược':' mượt ','muoc':' mượt ','muot':' mượt ','ms':' mà ','mây':' máy ',
        'nhung':' nhưng ',' nhưn ':' nhưng ',' ng ':' người ','nv':' nhân viên ','nt':' nhắn tin ','nhak':' nha ','nhậy':' nhạy ',
        'ô kêi': ' ok ', 'okie':' ok ',' o kê ':' ok ','okey':' ok ','ôkê':' ok ','oki':' ok ','oke':' ok ','okay':' ok ','okê' :' ok ','order':' đặt hàng ',
        'pó':' bó ','qá ':' qúa ',' pi ':' bị ','perfect':' rất tốt ',' pos ':'positive',
        ' r ':' rồi ','rùi':' rồi ','roi':' rồi ','rui':' rồi ',
        'sx':' sản xuất ','sam sung':' samsung ','ss':' samsung ','sài':' xài ','sd':' sử dụng ','sử dụg':' sử dụng ',
        'sp':' sản phẩm ','sấu':' xấu ','ship':' giao hàng ','sad':' tệ ','síp':' giao hàng ',
        'tks':' cám ơn ','thks':' cám ơn ','thanks':' cám ơn ','ths':' cám ơn ','thank ':' cám ơn ','thich':' thích ','thik':' thích ',' tes ':' test ',
        'tak':' ta ','tắc':' tắt ','tăc':' tắt ','tat':' tắt ',' tgd ':' thế giới di động ','tgdđ':' thế giới di động ','thegioididong':' thế giới di động ',
        'thay':' thấy ','tot':' tốt ','tra':' trả ','tut':' tụt ','tuộc':' tụt ','tuột':' tụt ','tục':' tụt ',' t ':' tôi ','tui':' tôi ','thiệc':'thiệt',' tl ':'trả lời',' trl ':'trả lời',
        ' up ':' update ',' updat ':' update ',' uong ':' uổng ',' vs ':' với ',' va ':' và ',' vẩn ':' vẫn ',' van ':' vẫn ',
        'wa':' qúa ','wá':' qúa ','wua':' qúa ','wed':' web ','wf':' wifi ',' wjfj ':' wifi ',
        'xau':' xấu ','xạc':' sạc ',' yt ':' youtube ','youtu ':' youtube ',' iu ':' yêu ',' zl ':' zalo ',
        }

In [18]:
class clean_data():
    def __init__(self,data):
        self.text =data
    def lower_case(self):
        text=self.text.lower()
        return text
    def translate_word(self):
        text = self.lower_case()
        text = re.sub(r'([A-Z])\1+', lambda m: m.group(1), text, flags=re.IGNORECASE)
        for k,v in replace_list.items():
            text=text.replace(k,v)
        return text
    def remove_special_word(self):
        text = self.translate_word()
        text=text.replace('●','')
        translator = str.maketrans(string.punctuation,' '*len(string.punctuation))
        return text.translate(translator)
    def segmentation(self):
        text = self.remove_special_word()
        text=ViTokenizer.tokenize(text)
        text =' '.join([i for i in text.split() if len(i)>=2])
        return text


In [33]:
data_after['comment'] = data_after['comment'].apply(lambda x: clean_data(x).segmentation())

In [34]:
data_after

,name,comment,star,source,label
0,Minh Tuấn,đã xài được năm cập_nhật đầy_đủ nên vẫn còn mư...,4,tgdd,1
1,Trtrnh,sử_dụng được 11 tháng cảm_giác máy khá ổn lâu_...,4,tgdd,1
2,Nguyễn Lương Kim Hảo,xài ổn pin trâu sạc nhanh camera chụp đẹp chơi...,5,tgdd,2
3,Minh,tôi sử_dụng gần năm thì sạc nhanh không họat đ...,3,tgdd,0
4,Nhật Huy,từ lúc máy nâng_cấp lên and rồi 10 là có hiện_...,1,tgdd,0
...,...,...,...,...,...
1690,Đtdd,điện thọai giống mô_tả tốt nhân_viên giao hàng...,5,tiki,2
1691,Tét He,nếu mua lọai này là auto đã được kích họat bảo...,5,tiki,2
1692,Nguyễn Nam Phong,sản_phẩm đẹp so với máy giá tầm trung mà hình ...,5,tiki,1
1693,Phạm Thị Thanh Diệu,có màu xanh không tiki ơi mình muốn mua màu xa...,4,tiki,1


In [35]:
data_before

,name,comment,star,source,label
0,Minh Tuấn,"Đã xài dc 1 năm, cập nhật đầy đủ nên vẫn còn m...",4,tgdd,1
1,Trtrnh,Sử dụng được 11 tháng. Cảm giác máy khá ổn. Lâ...,4,tgdd,1
2,Nguyễn Lương Kim Hảo,"Xài ổn, pin trâu sạc nhanh, camera chụp đẹp, c...",5,tgdd,2
3,Minh,Tôi sử dụng gần 1 năm thì sạc nhanh không hoạt...,3,tgdd,0
4,Nhật Huy,Từ lúc máy nâng cấp lên Android 10 là có hiện ...,1,tgdd,0
...,...,...,...,...,...
1690,Đtdd,"Điện thoại giống mô tả, tốt, nhân viên giao hà...",5,tiki,2
1691,Tét He,nếu mua loại này là auto đã được kích hoạt bảo...,5,tiki,2
1692,Nguyễn Nam Phong,"sản phẩm đẹp so với máy giá tầm trung, mà hình...",5,tiki,1
1693,Phạm Thị Thanh Diệu,có màu xanh k tiki ơi. mình muốn mua màu xanh ...,4,tiki,1


In [36]:
print(data_after.comment[1386])
print(data_after.comment[9])
print(data_after.comment[25])

kích họat bảo_hành dùm mình với tiki ơi
mua hồi tháng lúc giảm còn triệu máy ai cũng chê nhưng xài tháng rồi mọi thứ đều ổn nhân_viên tư vẫn nhiệt_tình tầm trung nên không_thể đòi_hỏi vân tay nhạy chơi game ít khi bị giật lag thiết_kế đẹp_mắt màu lưng máy siêu đẹp nói_chung những bạn có túi_tiền tầm_tầm thì vẫn nên mua rất tốt dành cho làm_việc giải_trí đều ok tin_dùng sa mà ung bao năm
máy dùng tác vụ thông_thường thì khá ổn nhưng pin hơi yếu vân tay rất bất_tiện khi bỏ điện_thoại trong túi quần vì bị chạm nên không dùng được


In [37]:
print(data_before.comment[1386])
print(data_before.comment[9])
print(data_before.comment[25])


Kích hoạt bảo hành dùm mình với Tiki ơiii .!!!!!!!!/
Mua hồi tháng 2 lúc giảm còn 5 triệu mấy .Ai cũng chê nhưng xài 1 tháng rồi mọi thứ đều ổn.Nhân viên tư vẫn nhiệt tình, tầm trung nên không thể đòi hỏi vân tay nhạy.Chs game ít khi bị giật lag.Thiết kế đẹp mắt.Màu lưng máy siêu đẹp.Nói chung những bạn có túi tiền tầm tầm thì vẫn nên mua rất tốt,dành cho làm việc giải trí đều ok.Tin dùng Samsung bao năm
Máy dùng tác vụ thông thường thì khá ổn nhưng pin hơi yếu , vân tay rất bất tiện khi bỏ điện thoại trong túi quần vì bị chạm nên không dùng được


In [38]:
data_after.to_csv(path_clean,index=False)